In [1]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [5]:
#llm.invoke("how can langsmith help with testing?")

'\nLangsmith is a tool that can help with testing in several ways:\n\n1. Automated Testing: Langsmith provides a built-in test framework that allows you to write and run automated tests for your code. This can save time and effort compared to writing and running tests by hand.\n2. Code Coverage Analysis: Langsmith can analyze the coverage of your tests, providing insights into which parts of your code are not being tested enough. This can help identify areas where you need to write more tests.\n3. Test Data Generation: Langsmith can generate test data automatically, based on the structure of your code. This can be especially useful for complex systems with many interdependent components.\n4. Code Refactoring: Langsmith can help you refactor your code by identifying areas where the tests are not covering enough code. This can help you to write more efficient and effective tests.\n5. Integration Testing: Langsmith provides integration testing features that allow you to test how different

In [2]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [3]:
chain = prompt | llm 

In [ ]:
#chain.invoke({"input":"what does ollama pull <model_name> do?"})

In [4]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [5]:
chain = prompt | llm | output_parser

In [ ]:
#chain.invoke({"input": "how can langsmith help with testing?"})

In [6]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

In [7]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

In [8]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = DocArrayInMemorySearch.from_documents(documents, embeddings)

In [10]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [11]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'\nBased on the provided context, langsmith can help with testing by allowing users to visualize their test results. This means that users can use langsmith to display and analyze their test data in a graphical format, which can help them better understand the results and make more informed decisions.'

In [12]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [14]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

LangSmith can help with testing in several ways:

1. Tracing: LangSmith provides detailed tracing of all LLM calls, which can help identify bottlenecks and issues in the chain or agent. By analyzing the traces, you can understand what is causing a slow response or unexpected output.
2. Debugging: LangSmith's tracing feature helps debug LLM calls by providing visibility into the exact inputs and outputs of each call. This makes it easier to identify bugs and errors in the chain or agent.
3. Testing: LangSmith provides a simple way to test LLMs, chains, and agents without requiring additional infrastructure. You can use LangSmith to generate test cases and evaluate the performance of your models under different conditions.
4. Monitoring: LangSmith allows you to monitor the performance of your LLMs, chains, and agents in real-time. This helps identify issues early on and enables you to take corrective action before they impact your users.
5. Collaborative debugging: LangSmith provides a c

In [15]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt) #retriever here is vectore store as a retriever

In [16]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='inputs, and see what happens. At some point though, our application is performing\nwell and we want to be more rigorous about testing changes. We can use a dataset\nthat we’ve constructed along the way (see above). Alternatively, we could spend some\ntime constructing a small dataset by hand. For these situations, LangSmith simplifies\ndataset uploading.Once we have a dataset, how can we use it to test changes to a prompt or chain? The most basic approach is to run the chain over the data points and visualize the outputs. Despite technological advancements, there still is no substitute for looking at outputs by eye. Currently, running the chain over the data points needs to be done client-side. The LangSmith client makes it easy to pull down a dataset and then run a chain over them, logging the results to a new project associated with the dataset. From there, you can review them. We\'ve made it easy to assign feedback to runs and mark them as correct or incorrec